<a href="https://colab.research.google.com/github/RohanOpenSource/Deep-Learning-And-Beyond/blob/main/ExplodingAndVanishingGradients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training neural networks can cause many problems that we can combat. The first of these is the exploding and vanishing gradient problem. During backpropogation, gradients can either become smaller and smaller, leaving to the lower layers of the neural network to remain unchanged, or they can become larger and larger making the neural network not work at all. This normally results in the usage of the sigmoid activation function or the normal distribution initlialization which both cause more variance between layers. To have this not happen, we need a similar variance for the inputs of each layer and the outputs. To acheive this, we can randomly initialize the weights. This is called Glorot and He Intilialization and is what is used by default in Keras.

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris() #you know the drill we're using iris
X =  iris.data[:, 2:]
y = iris.target

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="sigmoid", kernel_initializer=tf.keras.initializers.GlorotUniform), #sigmoid can also cause exploding/vanishing gradients
    tf.keras.layers.Dense(64, activation="relu"), #relu isn't perfect either as it leads to some neurons always outputting 0, thus being useless
    tf.keras.layers.Dense(32, activation="leaky_relu"), # rather than making negative numbers 0, leaky relu squashes them quite heavily meaning no neurons become useless
    tf.keras.layers.Dense(3, activation="softmax")
])

model.build([1, 2])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (1, 128)                  384       
_________________________________________________________________
dense_1 (Dense)              (1, 64)                   8256      
_________________________________________________________________
dense_2 (Dense)              (1, 32)                   2080      
_________________________________________________________________
dense_3 (Dense)              (1, 3)                    99        
Total params: 10,819
Trainable params: 10,819
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X, y, epochs=30)

Epoch 1/30
5/5 [==============================] - 1s 3ms/step - loss: 1.0754 - accuracy: 0.3667
Epoch 2/30
5/5 [==============================] - 0s 3ms/step - loss: 1.0288 - accuracy: 0.5000
Epoch 3/30
5/5 [==============================] - 0s 4ms/step - loss: 0.9890 - accuracy: 0.6667
Epoch 4/30
5/5 [==============================] - 0s 4ms/step - loss: 0.9422 - accuracy: 0.8333
Epoch 5/30
5/5 [==============================] - 0s 4ms/step - loss: 0.9018 - accuracy: 0.7800
Epoch 6/30
5/5 [==============================] - 0s 4ms/step - loss: 0.8420 - accuracy: 0.9333
Epoch 7/30
5/5 [==============================] - 0s 5ms/step - loss: 0.7749 - accuracy: 0.8133
Epoch 8/30
5/5 [==============================] - 0s 5ms/step - loss: 0.7200 - accuracy: 0.7733
Epoch 9/30
5/5 [==============================] - 0s 3ms/step - loss: 0.6567 - accuracy: 0.7400
Epoch 10/30
5/5 [==============================] - 0s 3ms/step - loss: 0.6021 - accuracy: 0.7533
Epoch 11/30
5/5 [======================

Sometimes, using variants of relu and glorot initialization is not enough to prevent the gradients from vanishing or exploding. In this case, batch normalization is required. Batch Normalization normalizes and centers the inputs of the layer before it. This reduces the variance between layers and if used as the first layer, can remove the need to normalize the data before hand.

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000] /  255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_4 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                3010      
Total params: 242,846
Trainable params: 240,678
Non-trainable params: 2,168
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.3742 - accuracy: 0.8625 - val_loss: 0.3315 - val_accuracy: 0.8758
Epoch 2/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.3297 - accuracy: 0.8797 - val_loss: 0.3127 - val_accuracy: 0.8866
Epoch 3/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.3020 - accuracy: 0.8873 - val_loss: 0.3118 - val_accuracy: 0.8922
Epoch 4/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.2822 - accuracy: 0.8954 - val_loss: 0.3245 - val_accuracy: 0.8830
Epoch 5/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.2632 - accuracy: 0.9014 - val_loss: 0.3085 - val_accuracy: 0.8890
Epoch 6/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.2510 - accuracy: 0.9072 - val_loss: 0.3299 - val_accuracy: 0.8800
Epoch 7/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.2389 - accuracy: 0.9112 - val_loss: 0.3218 - val_ac

Another way to avoid exploding/vanishing gradients is to clip the gradients before they become to big or small during backpropogation. This is called gradient clipping. 

In [ ]:
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss = "sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 7ms/step - loss: 0.1705 - accuracy: 0.9376 - val_loss: 0.2962 - val_accuracy: 0.9030
Epoch 2/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.1610 - accuracy: 0.9414 - val_loss: 0.2915 - val_accuracy: 0.9018
Epoch 3/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.1569 - accuracy: 0.9425 - val_loss: 0.2944 - val_accuracy: 0.9022
Epoch 4/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.1528 - accuracy: 0.9439 - val_loss: 0.2940 - val_accuracy: 0.9046
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.1511 - accuracy: 0.9454 - val_loss: 0.2924 - val_accuracy: 0.9032
